## Data loading

In [81]:
# Importing required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error as mse
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import statsmodels.api as sm
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings('ignore')


Load the test target data from the provided URL and convert the timestamp column to datetime format

In [2]:
# test target data URL: https://raw.githubusercontent.com/Jialunx/GEFCom/refs/heads/main/TestTar_W.csv

# hint
# test_target = pd.read_csv('...')
# test_target['TIMESTAMP'] = pd.to_datetime(test_target['TIMESTAMP'])
# test_target.head()

**Load the training and test data**

Steps to follow:
1. Initialize dictionaries to store training and test data
3. Load training data from URL
4. Convert timestamp column to datetime
5. Load test data from URL
6. Convert timestamp column to datetime
7. Merge test data with test_target on ZONEID and TIMESTAMP
 
(Note that test data URL does not include target value, hence, we merge "test_target" to test data)

In [4]:
# train data URL: https://raw.githubusercontent.com/Jialunx/GEFCom/refs/heads/main/Train_W_Zone1.csv
# test data URL: https://raw.githubusercontent.com/Jialunx/GEFCom/refs/heads/main/TestPred_W_Zone1.csv

# hint
# train_data = {}
# test_data = {}
# train_data = pd.read_csv(f'...')
# train_data['TIMESTAMP'] = pd.to_datetime(...)
# test_data = pd.read_csv(f'...')
# test_data['TIMESTAMP'] = pd.to_datetime(...)
# test_data = pd.merge(test_target,test_data)
# train_data.head()

•	**ZONEID**: Zone ID for each wind farm, ranging from 1 to 10. For this workshop, we only look at Zone 1.  
•	**TIMESTAMP**: Date and time for each recorded observation.  
•	**TARGETVAR**: The normalized power output of the wind farm during the given hour, ranging from 0 and 1.                     
•	**U10**: East-west component of the forecasted wind vector at 10 meters above ground level. Positive values indicate wind blowing eastward.  
•	**V10**: North-south component of the forecasted wind vector at 10 meters above ground level. Positive values indicate wind blowing northward.  
•	**U100**: East-west component of the forecasted wind vector at 100 meters above ground level. Positive values indicate wind blowing eastward.  
•	**V100**: North-south component of the forecasted wind vector at 100 meters above ground level. Positive values indicate wind blowing northward.  


## Data analysis

Check the relationship between the wind forecasts at 100 meters and the power output

In [27]:
# hint
# train_data.plot.scatter(x = '...',y = '...', c = '...', figsize = (...), colormap = '...', alpha = ...)

**Generate features**

Steps to follow:
1. Calculate wind speeds at 100m and 10m using U and V components
2. Calculate average wind speed between 100m and 10m
3. Calculate wind direction angles at both heights using arctan2
4. Combine training and test data for each zone into one dataframe

**wind speed at 100m:** ws100 = sqrt(U100^2 + V100^2)\
**wind speed at 10m:** ws10 = sqrt(U10^2 + V10^2) \
**average wind speed:** ws_avg = (ws100 + ws10)/2\
**wind direction at 100m:** angle100 = arctan2(V100, U100) * 180/pi\
**wind direction at 10m:** angle10 = arctan2(V10, U10) * 180/pi

In [5]:
# hint:
# train_data['ws100'] = np.sqrt(train_data['U100']**2 + train_data['V100']**2)
# train_data['ws10'] = ...
# train_data['ws_avg'] = ...
# train_data['polar_angle100'] = (180 / np.pi) * np.arctan2(train_data['V100'], train_data['U100'])
# train_data['polar_angle10'] = ...

# Same features for the test set
# test_data['ws100'] = ...
#   ...

# train_data.head()

**Plot the power generation and wind speed at 100m over time**

In [7]:
#hint
#fig, ax1 = plt.subplots(figsize=(12,3))
#ax2 = ax1.twinx()
#ins1 = ax1.plot(train_data['TARGETVAR'][:500], 'k', label='Wind Power')
#ins2 = ax2.plot(train_data[...][...], '...' , label=...)
#ax1.set_xlabel('Time')
#ax1.set_ylabel(...)
#ax2.set_ylabel(...)
#ax1.set_title(...)
#plt.grid(alpha=...)
#ax2.legend(ins1 + ins2, ['Wind Power', 'Wind Speed'])

**Using heatmap to plot the correlation between input features and target power generation for different zones**

Steps to follow:
1. Initialize an empty DataFrame for correlations
2. calculate correlations (drop 'ZONEID','TIMESTAMP' columns)

In [9]:
# hint
#corr_df = pd.DataFrame()
#corr_arr = train_data.drop(columns=[...]).corrwith(train_data['TARGETVAR'])
#corr_df = pd.concat([corr_df, corr_arr.rename(f'Power generation')], axis=1)
    
#fig, ax = plt.subplots(figsize=(3, 3))
#sns.heatmap(..., annot=True, ax=ax)
#plt.title('...')
#plt.show()

## Modelling

**Benchmark**

To evaluate prediction accuracy, it’s helpful to set a benchmark model. Here, the benchmark predicts the current month’s value as the average of the same month last year and the previous month.

Steps to follow:
1. Create a copy of training data and set timestamps as index
2.  - Get actual test values from previous data "test_target", which is from **'2013-12-01 01:00:00' to '2014-01-01 00:00:00'**
    - Estimate values by averaging same dates from:\
    **'2012-12-01 01:00:00' to '2013-01-01 00:00:00'**\
    **'2013-10-31 01:00:00' to '2013-12-01 00:00:00'**  
3. Compare actual vs estimated values using RMSE
4. Plot actual vs estimated

In [75]:
# Hint:
#temp_train = train_data
#temp_train = temp_train.set_index('TIMESTAMP')

#actual_values = test_data['...']
#estimated_values = ((temp_train['2012-12-01 01:00:00':'2013-01-01 00:00:00']['TARGETVAR'].reset_index(drop=True) + ...)/2)

#comparison_df = pd.DataFrame({'actual': actual_values, 'estimate': estimated_values})
#comparison_df.dropna(inplace=True)  #drop the nan values
#benchmark_rmse = np.sqrt(mse(..., ...))
#print(f'Benchmark RMSE is {benchmark_rmse:.4f}')  #(Benchmark RMSE is 0.2774)

#fig, ax = plt.subplots(figsize=(16,3))
#ax.plot(actual_values,color='r',label='Estimated Power Generation')
#...



**Multi-Layer Perceptron (MLP)**

In this notebook, we will use a single validation subset instead of cross-validation to reduce computational cost. The training data will be split into a training subset and a validation subset, with the validation subset comprising the most recent 10% of the data (approximately two months).\
The model's architecture, including the number of layers, neurons, and other hyperparameters, will be determined based on its performance on this validation set.

Steps to follow:
1. split training data into 90% train and 10% validation
2. rearrange train and validation into input and target\
     For input, keep all features for now (except TARGETVAR,  TIMESTAMP column)\
     For target, only keep TARGETVAR column

In [14]:
#hint
#split_limit = int(len(train_data) * 0.9)
#train = pd.concat([train_data.iloc[:split_limit, :]])
#eval = pd.concat([train_data.iloc[... , ...]])

#print(train.sample(5).to_string(index=True))

#def prepare_data(data, target_column='TARGETVAR', drop_column='TIMESTAMP'):
    #data = data.drop([drop_column], axis=1).dropna().reset_index(drop=True)
    #x = data.drop(target_column, axis=1).values
    #y = data[...].values
    #return x, y

#x_train, y_train = prepare_data(train)
#x_valid, y_valid = ...

Build 2-layer NN model with 64 neurons in each layer.\
To prevent overfitting, an early stoppage can be applied to stop the training process at a point when performance on a validation set decreases.

In [ ]:
# Define 2-layer NN model
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),  # First hidden layer
    layers.Dense(64, activation='relu'),  # Second hidden layer
    layers.Dense(1)  # Output layer
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)
# Add Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True, 
    verbose=1
)

model.summary()


In [ ]:
# Train the model
history = model.fit(
    x_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(x_valid, y_valid),
    callbacks=[early_stopping],  # Stop if no improvement
    verbose=1
)

# Make predictions on the validation set
predictions = model.predict(x_valid)

# Optional: Calculate RMSE manually for verification
rmse = np.sqrt(mse(y_valid, predictions))
print(f'Calculated validation RMSE: {rmse:.4f}')


The fast training time is due to relatively simple dataset (9 datapoint as input and 1 datapoint as output) and model structure. 

In [ ]:
# plot trainind and validation loss
train_loss = history.history['root_mean_squared_error']
val_loss = history.history['val_root_mean_squared_error']

plt.figure(figsize=(5, 3))
plt.plot(train_loss, label='Training Loss', linewidth=2)
plt.plot(val_loss, label='Validation Loss', linewidth=2)
plt.title('Training and Validation Loss', fontsize=10)
plt.xlabel('Epoch', fontsize=10)
plt.ylabel('Loss (RMSE)', fontsize=10)
plt.legend(fontsize=10)
plt.grid(True)
plt.show()

Use above trained model to check the model performance on test data. Plot the prediction result for test data and compare to benchmark model

In [21]:
#hint
#x_test, y_test = prepare_data(test_data)
#predictions = model.predict(...)
#rmse = np.sqrt(mse(..., ...))
#print(f'test RMSE: {rmse:.4f}')

#fig, ax = plt.subplots(figsize=(16,3))
#ax.plot(
#    actual_values.dropna().reset_index(drop=True), 
#    color='k', 
#    label='Actual Power Generation'
#)
#...
#ax.set_xlabel('Time [hour]')
#ax.set_ylabel('Power')
#ax.legend(loc='upper left')
#ax.set_title("Power Generation and Wind Speed over Time")
#plt.grid(linestyle='--', alpha=0.4)
#plt.show()

## Feature Engineering

1.	Drop **ZONEID** as input features as we only use 1 zone in this workshop, and it is expect that it does not contribute to the predictive power.
2.	Try remove or add other features to see the difference in RMSE for validation set


In [23]:
#def prepare_data(data, target_column='TARGETVAR', drop_columns=['TIMESTAMP', 'ZONEID', 'polar_angle100', 'polar_angle10']):  #try different features
#    data = data.drop(drop_columns, axis=1).dropna().reset_index(drop=True) 
#    x = data.drop(target_column, axis=1).values
#    y = data[target_column].values
#    return x, y

#x_train, y_train = prepare_data(train)
#x_valid, y_valid = prepare_data(eval)
#print(x_train.shape)    #check data shape
#print(y_train.shape)

#model = models.Sequential([
#    layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),  
#    layers.Dense(64, activation='relu'),  
#    layers.Dense(1)  
#])
#model.compile(
#    optimizer='adam',
#    loss='mse',
#    metrics=[tf.keras.metrics.RootMeanSquaredError()]
#)
#early_stopping = EarlyStopping(
#    monitor='val_loss', 
#    patience=20, 
#    restore_best_weights=True, 
#    verbose=0
#)
#history = model.fit(
#    x_train, y_train,
#    epochs=50,
#    batch_size=64,
#    validation_data=(x_valid, y_valid),
#    callbacks=[early_stopping],  
#    verbose=0
#)
#predictions = model.predict(x_valid)
#rmse = np.sqrt(mse(y_valid, predictions))
#print(f'Calculated validation RMSE: {rmse:.4f}')


In theory, incorporating the power output from the past few hours, along with forecasted changes in wind speed for the preceding and upcoming hours, could improve prediction accuracy. However, in practical applications, wind power forecasting is typically performed 24 to 48 hours ahead of real-time, making it impractical to use recent power output as an input. Instead, forecasted wind speeds from both previous and upcoming hours can be included as additional features to enhance the model's predictive performance. 

**Include the past wind speeds 'ws100' as the input to the model**

For the ease of the comparision, we can build a pipeline to streamline the process of evaluating different features and tuning hyperparameters. This will make it easier to compare performance across various configurations efficiently.

In [25]:
#hint
#def run_nn_pipeline(train_data, eval_data, target_column='TARGETVAR', drop_columns=['TIMESTAMP', 'ZONEID'], 
#                    hidden_units=64, activation='relu', optimizer='adam', loss='mse', 
#                    patience=10, epochs=50, batch_size=64, verbose=0):
    
# Prepare data 
#    def prepare_data(data, target_column, drop_columns):
#        data = data.drop(columns=drop_columns, errors='ignore').dropna().reset_index(drop=True)
#        x = data.drop(columns=[target_column], errors='ignore').values
#        y = data[target_column].values
#        return x, y

#    x_train, y_train = prepare_data(train_data, target_column, drop_columns)
#    x_valid, y_valid = prepare_data(eval_data, target_column, drop_columns)

    # Build model
#    model = models.Sequential([
#        layers.Dense(...),
#        layers.Dense(...),
#        layers.Dense(1)
#    ])

    # Compile and train
#    model.compile(optimizer=optimizer, loss=loss, metrics=['RootMeanSquaredError'])
    
#    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

#    model.fit(
#       x_train, y_train,
#       epochs=...
#       batch_size=...
#       validation_data=...
#       callbacks=...
#       verbose=...
#   )

    # Evaluate on test set
#    predictions = model.predict(x_valid)
#    rmse = np.sqrt(mse(y_valid, predictions))
#    print(f'Test RMSE: {rmse:.5f}')

#    return model, rmse, predictions


Add lag value of past 1 to 10 hours and check if the RMSE drops

In [27]:
#hint
#split_limit = int(len(train_data) * 0.9)
#train = pd.concat([train_data.iloc[:split_limit, :]])
#eval = pd.concat([train_data.iloc[split_limit:, :]])

#error = {}
#for lag in range(0,11):
#        train['ws100_lag{}'.format(lag)] = train['ws100'].shift(lag)
#        eval['ws100_lag{}'.format(lag)] = ...
#        model, rmse, predictions = run_nn_pipeline(
#                    train_data=train,
#                    eval_data=eval,
#                    target_column='TARGETVAR',
#                    drop_columns=['TIMESTAMP', 'ZONEID'],  # Drop additional columns if needed
#                    hidden_units=...,
#                    patience=...,
#                    epochs=...,
#                    batch_size=...
#        )
#        error[lag] = rmse

In [ ]:
#check the lag result
for i in range(0,11):
    print('RMSE for {:2} speed lag is {}'.format(i,error[i]))

lag_num = min(error, key=error.get)
print('The optimal number of speed lags to add is {} and the corresponding RMSE is {:.4}'
      .format(lag_num, error[lag_num]))

In [88]:
#Adding the optimal number of acceleration lags to the original set
split_limit = int(len(train_data) * 0.9)
train = pd.concat([train_data.iloc[:split_limit, :]])
eval = pd.concat([train_data.iloc[split_limit:, :]])
test = test_data

for lag in range(1,lag_num+1):
        train['ws100_lag{}'.format(lag)] = train['ws100'].shift(lag)
        eval['ws100_lag{}'.format(lag)] = eval['ws100'].shift(lag)
        test['ws100_lag{}'.format(lag)] = test['ws100'].shift(lag)

In [ ]:
# check data shape
print(train.shape)
print(eval.shape)
print(test.shape)
train.head()

## Fine-tuning

To procceed to hyperparameter tuning, we can further improve the **run_nn_pipeline** function to include number of layers and test data

In [94]:
def run_nn_pipeline(train_data, eval_data, test_data, target_column='TARGETVAR', drop_columns=['TIMESTAMP', 'ZONEID'], 
                    hidden_units=64, 
                    num_layers=2, # add num_layer
                    activation='relu', optimizer='adam', loss='mse', 
                    patience=10, epochs=50, batch_size=64, verbose=1, rmse_type='test'):
    
    # Prepare data
    def prepare_data(data, target_column, drop_columns):
        data = data.drop(columns=drop_columns, errors='ignore').dropna().reset_index(drop=True)
        x = data.drop(columns=[target_column], errors='ignore').values
        y = data[target_column].values
        return x, y

    x_train, y_train = prepare_data(train_data, target_column, drop_columns)
    x_valid, y_valid = prepare_data(eval_data, target_column, drop_columns)
    x_test, y_test = prepare_data(test_data, target_column, drop_columns)
#-------------------------------------

    model = models.Sequential()
    # Add the input layer
    model.add(layers.Dense(hidden_units, activation=activation, input_shape=(x_train.shape[1],)))
    # Add the hidden layers
    for _ in range(num_layers - 1):
        model.add(layers.Dense(hidden_units, activation=activation))
    # Add the output layer
    model.add(layers.Dense(1))
    
#--------------------------------------
 
    model.compile(optimizer=optimizer, loss=loss, metrics=['RootMeanSquaredError'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_valid, y_valid),
        callbacks=[early_stopping],
        verbose=verbose
    )

    # Evaluate on test set
    if rmse_type == 'validation':
        predictions = model.predict(x_valid)
        rmse = np.sqrt(mse(y_valid, predictions))
    elif rmse_type == 'test':
        predictions = model.predict(x_test)
        rmse = np.sqrt(mse(y_test, predictions))
    else:
        raise ValueError("Invalid rmse_type. Choose 'validation' or 'test'.")

    print(f'Calculated {rmse_type.capitalize()} RMSE: {rmse:.5f}')

    return model, rmse, predictions


In [ ]:
#hint
# #results = {}

# Iterate over layers and neurons
#for i in range(...):  # Number of layer
#    for j in range(...):  # Number of neurons (multiples of 32)
#        model, rmse, predictions = run_nn_pipeline(
#            train_data=train,
#            eval_data=eval,
#            test_data=test_data,
#            target_column='TARGETVAR',
#            drop_columns=['TIMESTAMP', 'ZONEID'], 
#            hidden_units=j,  
#            num_layers=i,  
#            activation='relu', 
#            optimizer='adam', 
#            loss='mse',  
#            patience=10,
#            epochs=50,
#            batch_size=64,
#            verbose=0,
#            rmse_type='validation'
#        )

        # Store RMSE for each combination of neurons and layers
#        results[(i, j)] = rmse

        # Print the result for the current combination
#        print(f"Number of layers: {i}, Number of neurons: {j}, RMSE: {rmse:.4f}")

# Find the combination with the minimum RMSE
#optimal_combination = min(results, key=results.get)
#optimal_neurons, optimal_layers = optimal_combination
#optimal_rmse = results[optimal_combination]

#print(f'\nThe optimal number of neurons = {optimal_layers}, layers = {optimal_neurons}, and the corresponding RMSE is {optimal_rmse:.4f}')


**Use the optimised parameter and train and test the model on test data**

In [ ]:
results = {}
# Iterate over layers and neurons

model, rmse, predictions = run_nn_pipeline(
            train_data=train,
            eval_data=eval,
            test_data=test,
            target_column='TARGETVAR',
            drop_columns=['TIMESTAMP'],  # Drop additional columns if needed
            hidden_units=128,  # Number of neurons
            num_layers=3,  # Number of layers
            activation='relu', 
            optimizer='adam', 
            loss='mse',  
            patience=10,
            epochs=50,
            batch_size=64,
            verbose=0,
            rmse_type='test'
        )

        # Store RMSE for each combination of neurons and layers
results = rmse

# Print the optimal result
print(f'\nThe test RMSE is {results:.4f}')


In [106]:
x_test, y_test = prepare_data(test, 'TARGETVAR', ['TIMESTAMP'])
prediction = model(x_test)  # ML prediction on test data

**Compare the prediction result to the theoretical model**

Using the provided operational curve, interpolate the curve to estimate the power output for the wind speeds in the test data. Assume the wind farm has an operational capacity of 400 MW, based on 40 wind turbines. Scale both the theoretical predictions from the curve and the ML model predictions to 400 MW for comparison

In [ ]:
# Wind speed (U) in m/s
U = [4, 5, 6, 7, 8, 9, 10, 11, 11.17, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
# Power output (MW) 
Power = [0.459, 0.897, 1.55, 2.462, 3.674, 5.232, 7.177, 9.552, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

plt.figure(figsize=(5,3))
plt.plot(U, Power, 'k', linewidth=2)
plt.grid(True)
plt.xlabel('Wind Speed [m/s]')
plt.ylabel('Power Output [MW]')
plt.title('Operational curve')
plt.show()

In [112]:
# hint
# can use interp1d function to interpolate power output for given wind speeds

#scaling = 400 
#test = test.dropna().reset_index(drop=True)
#actual_values = test['TARGETVAR'] * scaling
#wind_speed = test['ws100']

# Interpolate power output for given wind speeds with extrapolation
#interpolated_power = ... * scaling / 10


# Scaling the predicted values
#scaled_prediction = prediction * scaling

#fig, ax = plt.subplots(figsize=(12, 3))
#...

# Calculating RMSE for ML and theoretical predictions
#rmse_ML = np.sqrt(mse(scaled_prediction, actual_values))/scaling
#rmse_theory = np.sqrt(mse(interpolated_power, actual_values))/scaling

# Printing the RMSE
#print(f'ML RMSE is {rmse_ML:.4f}')
#print(f'Theoretical RMSE is {rmse_theory:.4f}')
